# True DiT on CIFAR (did not run)

# Imports

In [ ]:
%%capture
!pip install einops
# !pip install torchinfo

In [ ]:
import json
import time

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.utils as vutils

from IPython.display import HTML

# from torchinfo import summary

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from constants import *
from model import AdaLNDiT
from utils import set_seed, load_data, drop_label, device
from train import (
    load_model_optim_loss_imgs,
    train,
    generate_single,
    generate_all_classes,
    generate_all_classes_steps,
    generate_uncond,
    interpolate,
    interpolate2,
)

DATASET = "CIFAR-10"
MODEL_NAME = f"{N_LAYER}_LAYERs_{N_HEAD}_HEADs_{N_EMBD}_EMBD_DIM_{T_MAX}_TMAX_{DATASET}"
MODEL_NAME = "ada_ln_dit_cfg_" + MODEL_NAME
PATEL = False  ## use Patel CFG loss for training
if PATEL:
    TRAIN_GUIDANCE = 1
    SAMPLE_GUIDANCE = 3

# Data

In [ ]:
train_data, val_data, train_loader, val_loader = load_data(DATASET)

# Initialization

In [ ]:
set_seed()
model = AdaLNDiT(
    N_EMBD,
    N_HEAD,
    # N_KV_HEAD,
    N_FF,
    N_LAYER,
    n_class=N_CLASS,
    n_channel=IMG_CHANNELS,
    img_size=IMG_SIZE,
    patch_size=PATCH_SIZE,
    t_max=T_MAX,
    beta_min=BETA_MIN,
    beta_max=BETA_MAX,
    schedule=SCHEDULE,
    s=0.008,
    norm_first=True,
    device=device,
    dropout=0,
)

model.to(device)
# summary(model)

criterion = nn.MSELoss()

In [ ]:
x = torch.randn(BATCH_SIZE, IMG_CHANNELS, IMG_SIZE, IMG_SIZE).to(device)
t = torch.randint(T_MAX, (BATCH_SIZE,)).to(device)
label = torch.full((BATCH_SIZE,), LABEL).to(device)

print(model(x, t, label).shape)  # (B, IMG_CHANNELS, IMG_SIZE, IMG_SIZE)

torch.Size([64, 1, 28, 28])


In [ ]:
print(f"Number of model parameters: {sum(p.numel() for p in model.parameters())}")

Number of model parameters: 9718032


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# ––––––––––––––––––––

# Train

# Driver code

In [ ]:
if CHECKPOINT:
    train_losses, img_list = load_model_optim_loss_imgs(model, optimizer, MODEL_NAME)
    train(
        model,
        train_loader,
        criterion,
        optimizer,
        device,
        MODEL_NAME,
        PATEL,
        TRAIN_GUIDANCE,
        SAMPLE_GUIDANCE,
        train_losses=train_losses,
        img_list=img_list,
    )
else:
    train(
        model,
        train_loader,
        criterion,
        optimizer,
        device,
        MODEL_NAME,
        PATEL,
        TRAIN_GUIDANCE,
        SAMPLE_GUIDANCE,
    )

# Loss curve

In [ ]:
with open(f"{path}/train_logs/{MODEL_NAME}_train_losses.json", "r") as f:
    train_losses = json.load(f)

plt.figure(figsize=(8, 5))
plt.plot(train_losses[1:])  # skip very first loss
plt.xlabel(f"Iterations (x {PRINT_EVERY})")
plt.ylabel("Train Loss")
plt.show()

# Visualization of diffusion

In [ ]:
img_list = torch.load(f"{path}/train_logs/{MODEL_NAME}_img_list.pt")

### Animation

In [ ]:
fig = plt.figure(figsize=(8, 8))  # assuming batch size 64
plt.axis("off")
ims = [
    [plt.imshow(img.permute(1, 2, 0).cpu().numpy(), animated=True)] for img in img_list
]
anim = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(anim.to_jshtml())

### Side by side comparison

In [ ]:
real_img, _ = next(iter(train_loader))

# Plot the real images
plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(
    np.transpose(
        vutils.make_grid(real_img.to(device), padding=5, normalize=True).cpu(),
        (1, 2, 0),
    )
)

# Plot the fake images from the last saved img_list
plt.subplot(1, 2, 2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(img_list[-1].permute(1, 2, 0).cpu().numpy())
plt.show()